# patient-online-deprivation-score-model
#### Developed by: Mary Amanuel
#### Contact: mary.amanuel@nhsx.nhs.uk
#### Created: 22nd November 2021
#### Last updated: 22nd November 2021


In [73]:
import pandas as pd
import os
import plotly
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json

c
##### 'Public Health England. Public Health Profiles. [22/11/2021] https://fingertips.phe.org.uk © Crown copyright [2021]'
##### Technical documentation for data here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833951/IoD2019_Technical_Report.pdf

##### Research documentation for data here: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833947/IoD2019_Research_Report.pdf

In [74]:
# Import fingertips_py package - This is a python package to interact with Public Health England’s [Fingertips](https://fingertips.phe.org.uk/) data tool. Fingertips is a major repository of population and public health indicators for England.
## https://fingertips-py.readthedocs.io/en/latest/#indices-and-table
## Installing the fingertips_py library (if not already)

!pip install fingertips_py
import fingertips_py

In [75]:
national_gp_profiles = fingertips_py.get_profile_by_name('National General Practice Profiles')
national_gp_profiles

{'Id': 21,
 'Name': 'National General Practice Profiles (supporting indicators)',
 'Key': 'practice-profiles-supporting-indicators',
 'GroupIds': [1200006],
 'GroupMetadata': [{'Id': 1200006,
   'Name': 'National GP Profiles - Supporting Indicators',
   'Sequence': 1,
   'ProfileId': 21,
   'DataAvailable': [{'AreaTypeId': 7, 'IndicatorCount': 20},
    {'AreaTypeId': 165, 'IndicatorCount': 20},
    {'AreaTypeId': 166, 'IndicatorCount': 20},
    {'AreaTypeId': 167, 'IndicatorCount': 20},
    {'AreaTypeId': 204, 'IndicatorCount': 20},
    {'AreaTypeId': 219, 'IndicatorCount': 20}]}],
 'Visualisations': ['Overview',
  'CompareAreas',
  'Trends',
  'AreaProfile',
  'Definitions',
  'Download'],
 'HasRecentTrends': False,
 'StartValueAxisFromZero': False,
 'IsChangeFromPreviousPeriodShown': False,
 'DefaultAreaTypeId': 7,
 'DefaultTabId': 0,
 'IgnoredSpineChartAreas': ['E06000053', 'E09000001'],
 'OpenInequalitiesOnAreaData': False}

In [76]:
## Check the relevant geographies that can be called upon as area id. For instance, it can be seen that 'General Practice' is 7
fingertips_py.metadata.get_area_types_as_dict()

[{'Name': 'Middle Super Output Area',
  'Short': 'MSOA',
  'Class': None,
  'CanBeDisplayedOnMap': True,
  'Id': 3},
 {'Name': 'Lower Super Output Area',
  'Short': 'LSOA',
  'Class': None,
  'CanBeDisplayedOnMap': False,
  'Id': 4},
 {'Name': 'Government Office Region',
  'Short': 'Region',
  'Class': None,
  'CanBeDisplayedOnMap': True,
  'Id': 6},
 {'Name': 'General Practice',
  'Short': 'GP',
  'Class': None,
  'CanBeDisplayedOnMap': True,
  'Id': 7},
 {'Name': 'Ward',
  'Short': 'Ward',
  'Class': None,
  'CanBeDisplayedOnMap': True,
  'Id': 8},
 {'Name': 'Acute Trust',
  'Short': 'Acute Trust',
  'Class': None,
  'CanBeDisplayedOnMap': False,
  'Id': 14},
 {'Name': 'England',
  'Short': 'England',
  'Class': None,
  'CanBeDisplayedOnMap': False,
  'Id': 15},
 {'Name': 'Mental Health Trust',
  'Short': 'Mental Health Trust',
  'Class': None,
  'CanBeDisplayedOnMap': False,
  'Id': 20},
 {'Name': 'Ambulance Trust',
  'Short': 'Ambulance Trust',
  'Class': None,
  'CanBeDisplayedOnM

In [77]:
# Obtain dataframe with metadata on metrics under chosen topic (can instead just navigate website direct)
national_gp_profiles_meta = fingertips_py.get_metadata_for_profile_as_dataframe(national_gp_profiles['Id'])
national_gp_profiles_meta

,Indicator ID,Indicator,Definition,Rationale,Data source,Indicator source,Methodology,Standard population/values,Confidence interval details,Source of numerator,...,Notes,Frequency,Rounding,Indicator Content,Specific rationale,Impact of COVID-19,Unit,Value type,Year type,Polarity
0,337,GP registered population by sex and quinary ag...,Number of males and females registered with a ...,NaN,NHS digital,Data for April of the respective calendar year...,NaN,NaN,NaN,NaN,...,Data are extracted as monthly snapshot in time...,NaN,NaN,NaN,NaN,NaN,NaN,Count,Calendar,Not applicable
1,114,QOF Total List Size,Total number of patients registered with the p...,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",Data for 2019/20 can be downloaded from https...,NaN,NaN,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Count,Financial,Not applicable
2,295,Total QOF points,The percentage of all QOF points achieved acro...,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",Data for 2019/20 can be downloaded from https:...,NaN,NaN,NaN,"Quality and Outcomes Framework (QOF), NHS Digital",...,The maximal number of QOF points in 2013/14 wa...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial,RAG - High is good
3,650,Life expectancy - MSOA based,Life expectancy at birth. Population weighting...,Life expectancy at birth is chosen as the pref...,"PHE, based on ONS mortality data",http://www.localhealth.org.uk for MSOA life ex...,The life expectancy for the MSOAs where the pr...,NaN,NaN,NaN,...,This indicator was initially calculated for MS...,NaN,NaN,NaN,NaN,NaN,Years,Life expectancy,Calendar,RAG - High is good
4,93438,% who have a positive experience of their GP p...,"Question 30. People were asked: ""Overall, how ...",NaN,GP Patient Survey,http://www.gp-patient.co.uk,NaN,NaN,NaN,NaN,...,The number of this question has changed from 3...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Calendar,BOB - Blue orange blue
5,1679,Ethnicity estimates,Estimated proportion of non-white ethnic group...,NaN,Census 2011(Ethnicity by LSOA); HSCIC: Numbers...,Department of Primary Care and Public Health S...,NaN,NaN,NaN,NaN,...,If an ethnic group's proportion is smaller tha...,NaN,NaN,NaN,NaN,NaN,%,Proportion,Financial,Not applicable
6,93558,IDACI - Income Deprivation Affecting Children ...,This is a subset of the Income Deprivation Dom...,The Income Deprivation Affecting Children Inde...,"Ministry of Housing, Communities & Local Gover...",IDAOPI and IDACI are part of the English Indic...,Where area values have been published in the d...,NaN,NaN,NaN,...,NaN,The indices of deprivation are published in ir...,NaN,NaN,NaN,NaN,NaN,Proportion,Calendar,RAG - Low is good
7,93559,IDAOPI - Income Deprivation Affecting Older Pe...,This is a subset of the Income Deprivation Dom...,Deprivation is one of the strongest predictors...,"Ministry of Housing, Communities & Local Gover...",The IDACI and IDAOPI are part of the English I...,Where area values have been published in the d...,NaN,NaN,NaN,...,NaN,The indices of deprivation are published in ir...,NaN,NaN,NaN,NaN,NaN,Proportion,Calendar,RAG - Low is good
8,93553,Deprivation score (IMD 2019),The English Indices of Deprivation 2019 use 39...,Deprivation covers a broad range of issues and...,"Ministry of Housing, Communities & Local Gover...",https://www.gov.uk/government/statistics/engli...,The English Indices of Deprivation 2019 are me...,NaN,NaN,NaN,...,The Indices of Deprivation 2019 are relative m...,NaN,NaN,NaN,NaN,NaN,NaN,Score,Calendar,RAG - Low is good


In [78]:
gp_and_england_deprivation_score = fingertips_py.retrieve_data.get_data_by_indicator_ids(indicator_ids=[93553],area_type_id =[7] ) # 7 for practice (we know this topic area has practice breakdown for sure, by design), 93553 for our metric of interest
gp_and_england_deprivation_score

,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,...,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Column not used,Time period Sortable,New data,Compared to goal
0,93553,Deprivation score (IMD 2019),NaN,NaN,E92000001,England,England,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
1,93553,Deprivation score (IMD 2019),E92000001,England,A81001,The Densham Surgery,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
2,93553,Deprivation score (IMD 2019),E92000001,England,A81002,Queens Park Medical Centre,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
3,93553,Deprivation score (IMD 2019),E92000001,England,A81004,Bluebell Medical Centre,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
4,93553,Deprivation score (IMD 2019),E92000001,England,A81005,Springwood Surgery,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,93553,Deprivation score (IMD 2019),E92000001,England,Y05857,RHND,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
7604,93553,Deprivation score (IMD 2019),E92000001,England,Y06007,Wellbourne Healthcare CIC,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
7605,93553,Deprivation score (IMD 2019),E92000001,England,Y06113,QHS GP Care Home Service,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN
7606,93553,Deprivation score (IMD 2019),E92000001,England,Y06218,Brownsover Medical Centre,GP,Persons,All ages,NaN,...,NaN,NaN,NaN,NaN,Cannot be calculated,Not compared,Not compared,20190000,NaN,NaN


In [79]:
gp_and_england_deprivation_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7608 entries, 0 to 7607
Data columns (total 26 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Indicator ID                              7608 non-null   int64  
 1   Indicator Name                            7608 non-null   object 
 2   Parent Code                               7607 non-null   object 
 3   Parent Name                               7607 non-null   object 
 4   Area Code                                 7608 non-null   object 
 5   Area Name                                 7608 non-null   object 
 6   Area Type                                 7608 non-null   object 
 7   Sex                                       7608 non-null   object 
 8   Age                                       7608 non-null   object 
 9   Category Type                             0 non-null      float64
 10  Category                            

In [90]:
gp_and_england_deprivation_score = gp_and_england_deprivation_score[['Area Code', 'Area Name', 'Area Type', 'Time period', 'Value']]
gp_and_england_deprivation_score

,Area Code,Area Name,Area Type,Time period,Value
0,E92000001,England,England,2019,21.722087
1,A81001,The Densham Surgery,GP,2019,32.360939
2,A81002,Queens Park Medical Centre,GP,2019,31.921958
3,A81004,Bluebell Medical Centre,GP,2019,28.024026
4,A81005,Springwood Surgery,GP,2019,15.398207
...,...,...,...,...,...
7603,Y05857,RHND,GP,2019,23.386000
7604,Y06007,Wellbourne Healthcare CIC,GP,2019,44.559787
7605,Y06113,QHS GP Care Home Service,GP,2019,32.817919
7606,Y06218,Brownsover Medical Centre,GP,2019,15.623656


In [91]:
gp_deprivation_score_2019 = gp_and_england_deprivation_score[gp_and_england_deprivation_score['Area Type'].isin(['GP'])]
gp_deprivation_score_2019 = gp_deprivation_score_2019.rename(columns={'Area Code' :'Area_Code', 
                                          'Area Name' : 'Area_Name',
                                          'Area Type' : 'Area_Type',
                                          'Time period' : 'Time_Period', 
                                          'Value' : 'Deprivation_Score_IMD_2019'})
gp_deprivation_score_2019['Area_Name'] = gp_deprivation_score_2019['Area_Name'].str.upper()

In [92]:
gp_deprivation_score_2019

,Area_Code,Area_Name,Area_Type,Time_Period,Deprivation_Score_IMD_2019
1,A81001,THE DENSHAM SURGERY,GP,2019,32.360939
2,A81002,QUEENS PARK MEDICAL CENTRE,GP,2019,31.921958
3,A81004,BLUEBELL MEDICAL CENTRE,GP,2019,28.024026
4,A81005,SPRINGWOOD SURGERY,GP,2019,15.398207
5,A81006,TENNANT STREET MEDICAL PRACTICE,GP,2019,35.612387
...,...,...,...,...,...
7603,Y05857,RHND,GP,2019,23.386000
7604,Y06007,WELLBOURNE HEALTHCARE CIC,GP,2019,44.559787
7605,Y06113,QHS GP CARE HOME SERVICE,GP,2019,32.817919
7606,Y06218,BROWNSOVER MEDICAL CENTRE,GP,2019,15.623656


In [93]:
gp_deprivation_score_2019.describe()

,Time_Period,Deprivation_Score_IMD_2019
count,6980.0,6980.000000
mean,2019.0,23.530374
std,0.0,11.666802
min,2019.0,3.404973
25%,2019.0,14.089732
50%,2019.0,21.576292
75%,2019.0,30.878554
max,2019.0,68.721015


In [97]:
gp_deprivation_score_2019['GP_Rank'] = gp_deprivation_score_2019['Deprivation_Score_IMD_2019'].rank()
gp_deprivation_score_2019

,Area_Code,Area_Name,Area_Type,Time_Period,Deprivation_Score_IMD_2019,GP_Rank
1,A81001,THE DENSHAM SURGERY,GP,2019,32.360939,5461.0
2,A81002,QUEENS PARK MEDICAL CENTRE,GP,2019,31.921958,5394.0
3,A81004,BLUEBELL MEDICAL CENTRE,GP,2019,28.024026,4741.0
4,A81005,SPRINGWOOD SURGERY,GP,2019,15.398207,2097.0
5,A81006,TENNANT STREET MEDICAL PRACTICE,GP,2019,35.612387,5861.0
...,...,...,...,...,...,...
7603,Y05857,RHND,GP,2019,23.386000,3873.0
7604,Y06007,WELLBOURNE HEALTHCARE CIC,GP,2019,44.559787,6565.0
7605,Y06113,QHS GP CARE HOME SERVICE,GP,2019,32.817919,5531.0
7606,Y06218,BROWNSOVER MEDICAL CENTRE,GP,2019,15.623656,2149.0


In [101]:
gp_deprivation_score_2019['GP_Rank'] = gp_deprivation_score_2019['Deprivation_Score_IMD_2019'].rank(ascending = False)
gp_deprivation_score_2019

,Area_Code,Area_Name,Area_Type,Time_Period,Deprivation_Score_IMD_2019,GP_Rank
1,A81001,THE DENSHAM SURGERY,GP,2019,32.360939,1520.0
2,A81002,QUEENS PARK MEDICAL CENTRE,GP,2019,31.921958,1587.0
3,A81004,BLUEBELL MEDICAL CENTRE,GP,2019,28.024026,2240.0
4,A81005,SPRINGWOOD SURGERY,GP,2019,15.398207,4884.0
5,A81006,TENNANT STREET MEDICAL PRACTICE,GP,2019,35.612387,1120.0
...,...,...,...,...,...,...
7603,Y05857,RHND,GP,2019,23.386000,3108.0
7604,Y06007,WELLBOURNE HEALTHCARE CIC,GP,2019,44.559787,416.0
7605,Y06113,QHS GP CARE HOME SERVICE,GP,2019,32.817919,1450.0
7606,Y06218,BROWNSOVER MEDICAL CENTRE,GP,2019,15.623656,4832.0


In [102]:
gp_deprivation_score_2019_ranking = gp_deprivation_score_2019.sort_values(by='GP_Rank', ascending=True)
gp_deprivation_score_2019_ranking

,Area_Code,Area_Name,Area_Type,Time_Period,Deprivation_Score_IMD_2019,GP_Rank
6501,N82099,MERE LANE GROUP PRACTICE,GP,2019,68.721015,1.0
62,A81621,THORNTREE SURGERY,GP,2019,67.407408,2.0
6530,N82669,GREAT HOMER STREET MEDICAL CENTRE,GP,2019,65.248605,3.0
6502,N82101,KIRKDALE,GP,2019,65.104433,4.0
6523,N82651,STANLEY MEDICAL CENTRE,GP,2019,64.957030,5.0
...,...,...,...,...,...,...
4968,K82049,HUGHENDEN VALLEY SURGERY,GP,2019,4.174575,6976.0
4856,K81028,MAGNOLIA HOUSE SURGERY,GP,2019,4.158261,6977.0
4662,J82099,RICHMOND SURGERY,GP,2019,4.151031,6978.0
4899,K81092,WOOSEHILL PRACTICE,GP,2019,3.933291,6979.0


### Patient Online Data 2019